In [1]:
import re
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# تحميل الموديل والتوكنيزر
model_name = "abdalrahmanshahrour/AraBART-summ"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# الأمثلة
texts = [
    "أعلنت وزارة التعليم في مؤتمر صحفي عن خطة جديدة تهدف إلى تطوير المناهج الدراسية في جميع المراحل التعليمية، وذلك ضمن جهودها لتحسين جودة التعليم وزيادة كفاءة الطلاب.",
    "أصدرت وزارة الصحة تقريرًا جديدًا حول حالات الإصابة بفيروس كورونا والإجراءات المتخذة لمواجهته."
]

references = [
    "وزارة التعليم تعلن خطة لتطوير المناهج في كل المراحل ضمن تحسين جودة التعليم.",
    "تقرير جديد من وزارة الصحة عن حالات كورونا والإجراءات المتخذة."
]

# تلخيص النصوص
predictions = []

for text in texts:
    input_text = "summarize: " + text  # بدون تنظيف متقدم
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=512)

    summary_ids = model.generate(
        inputs["input_ids"],
        max_length=80,
        min_length=15,
        num_beams=6,
        length_penalty=1.2,
        no_repeat_ngram_size=3,
        early_stopping=True
    )

    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    predictions.append(summary)

# دالة تنظيف بسيطة قبل التقييم
def clean_for_eval(text):
    text = re.sub(r'[^\w\s]', '', text)  # إزالة علامات الترقيم
    return text.strip()

# دالة لحساب "accuracy بسيطة"
def simple_accuracy(predictions, references):
    matched = 0
    total = len(predictions)
    for pred, ref in zip(predictions, references):
        pred_words = set(clean_for_eval(pred).split())
        ref_words = set(clean_for_eval(ref).split())
        if len(pred_words & ref_words) > 0:
            matched += 1
    return matched / total

# عرض النتائج
print("🔹 التلخيصات المتوقعة:")
for pred in predictions:
    print("➡️", pred)

acc = simple_accuracy(predictions, references)
print(f"\n✅ Simple Accuracy: {acc:.2%}")




/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/557M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/557M [00:00<?, ?B/s]

🔹 التلخيصات المتوقعة:
➡️ أعلنت وزارة التعليم في المملكة العربية السعودية عن خطة جديدة تهدف إلى تطوير المناهج الدراسية في جميع المراحل التعليمية.
➡️ أصدرت وزارة الصحة السعودية تقريراا جديداا حول حالات الإصابة بفيروس كورونا والإجراءات المتخذة لمواجهته.

✅ Simple Accuracy: 100.00%
